In [1]:
%pwd

'c:\\Users\\moina\\Downloads\\chatapp\\backend\\research'

In [ ]:
print("ok")

In [2]:
import os
os.chdir('..')

In [17]:
%pwd

'c:\\Users\\moina\\Downloads\\med-chatbot'

In [14]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [18]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [19]:
extracted_data=load_pdf_file(data='Data/')

In [ ]:
extracted_data

In [20]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [21]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [22]:
from langchain.embeddings import HuggingFaceEmbeddings

In [23]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()

C:\Users\moina\AppData\Local\Temp\ipykernel_24860\220312833.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [24]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [25]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [ ]:
%pwd 

In [26]:

PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
NVIDIA_API_KEY=os.environ.get('NVIDIA_API_KEY')

In [ ]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["NVIDIA_API_KEY"] = NVIDIA_API_KEY

In [27]:
from pinecone import Pinecone

pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [28]:
from pinecone import Pinecone, ServerlessSpec
import os

# Get API key from environment
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

index_name = "medical-bot"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [29]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [30]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [31]:
docsearch

In [32]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [33]:
retrived_docs = retriver.get_relevant_documents("What is the Acne")

C:\Users\moina\AppData\Local\Temp\ipykernel_24860\2768571858.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrived_docs = retriver.get_relevant_documents("What is the Acne")


In [34]:
retrived_docs

[Document(id='807288c9-37ea-400a-8316-d52bc03c5213', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='26c00228-2648-4e33-a4c2-e15a102e7521', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='5943dd2f-8cf4-4f8a-8501-8a34f695039e', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 

In [36]:
from langchain.openai import OpenAI

llm = OpenAI(temperature=0.4, max_tokens=500)

ModuleNotFoundError: No module named 'langchain.openai'

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:

question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:

response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

In [35]:
from typing import Any, List, Optional
from langchain_core.language_models.llms import LLM
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.outputs import LLMResult
from openai import OpenAI
import os


# 1. Create NVIDIA client
nvidia_client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.getenv("NVIDIA_API_KEY")
)


# 2. Wrap NVIDIA into a LangChain-compatible LLM
class NvidiaLLM(LLM):
    model: str = "openai/gpt-oss-120b"
    temperature: float = 0.4
    max_tokens: int = 500

    @property
    def _llm_type(self) -> str:
        return "nvidia-llm"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        completion = nvidia_client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            max_tokens=self.max_tokens,
        )
        return completion.choices[0].message["content"]

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> LLMResult:
        responses = [self._call(p, stop=stop, run_manager=run_manager, **kwargs) for p in prompts]
        return LLMResult(generations=[[{"text": r}] for r in responses])


# 3. Use it instead of OpenAI
chatModel = NvidiaLLM()

# 4. Keep the rest of your RAG chain the same
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])


NameError: name 'retriever' is not defined

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# --- PDF Loading ---
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    return loader.load()

extracted_data = load_pdf_file(data='Data/')

# --- Text Splitting ---
def text_split(extracted_data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    return splitter.split_documents(extracted_data)

text_chunks = text_split(extracted_data)

# --- Embeddings ---
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# --- Pinecone Setup ---
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "medical-bot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# --- NVIDIA LLM wrapper ---
from typing import Any, List, Optional
from langchain_core.language_models.llms import LLM
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.outputs import LLMResult
from openai import OpenAI

nvidia_client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.getenv("NVIDIA_API_KEY")
)

class NvidiaLLM(LLM):
    model: str = "openai/gpt-oss-120b"
    temperature: float = 0.4
    max_tokens: int = 500

    @property
    def _llm_type(self) -> str:
        return "nvidia-llm"

    def _call(
        self, prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        completion = nvidia_client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            max_tokens=self.max_tokens,
        )
        return completion.choices[0].message["content"]

    def _generate(
        self, prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> LLMResult:
        responses = [self._call(p, stop=stop, run_manager=run_manager, **kwargs) for p in prompts]
        return LLMResult(generations=[[{"text": r}] for r in responses])

chatModel = NvidiaLLM()

# --- Prompt & RAG Chain ---
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# --- Test Query ---
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])


In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.getenv("NVIDIA_API_KEY")
)

resp = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[{"role": "user", "content": "Say hello from NVIDIA"}],
    max_tokens=50,
)
print(resp.choices[0].message["content"])


In [ ]:
import os
print(os.getenv("NVIDIA_API_KEY"))


In [ ]:
import requests, os

headers = {"Authorization": f"Bearer {os.getenv('NVIDIA_API_KEY')}"}
url = "https://integrate.api.nvidia.com/v1/status"  # or their recommended test endpoint
r = requests.get(url, headers=headers)
print(r.status_code, r.text)


In [ ]:
pip install python-dotenv langchain langchain-nvidia-ai-endpoints pypdf sentence-transformers pinecone-client langchain-pinecone


In [ ]:
# ----------------------------
# Load environment variables
# ----------------------------
from dotenv import load_dotenv
import os

load_dotenv()  # loads NVIDIA_API_KEY and PINECONE_API_KEY from .env

NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# ----------------------------
# PDF Loading
# ----------------------------
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

def load_pdf_files(data_path):
    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    return loader.load()

documents = load_pdf_files("Data/")  # replace with your folder

# ----------------------------
# Text Splitting
# ----------------------------
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(docs):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    return splitter.split_documents(docs)

text_chunks = split_text(documents)

# ----------------------------
# Embeddings
# ----------------------------
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ----------------------------
# Pinecone Setup
# ----------------------------
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medical-bot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# ----------------------------
# NVIDIA LLM Setup
# ----------------------------
from langchain_nvidia_ai_endpoints import ChatNVIDIA

nvidia_llm = ChatNVIDIA(
    model="openai/gpt-oss-120b",
    api_key=NVIDIA_API_KEY,
    temperature=0.4,
    top_p=1,
    max_tokens=500
)

# ----------------------------
# Prompt Template
# ----------------------------
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following retrieved context to answer the question. "
    "If you don't know the answer, say so. "
    "Keep the answer concise, max three sentences.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(nvidia_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# ----------------------------
# Test Query
# ----------------------------
response = rag_chain.invoke({"input": "What is Acromegaly and gigantism?"})
print(response["answer"])


C:\Users\moina\AppData\Local\Temp\ipykernel_24860\3455666486.py:66: DeprecationWarning: The 'max_tokens' parameter is deprecated and will be removed in a future version. Please use 'max_completion_tokens' instead.
  nvidia_llm = ChatNVIDIA(


In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
print("PINECONE_API_KEY:", os.getenv("PINECONE_API_KEY"))

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from parent directory
load_dotenv(dotenv_path='../.env')

print("Current directory:", os.getcwd())
print("PINECONE_API_KEY loaded:", bool(os.getenv("PINECONE_API_KEY")))

try:
    from pinecone import Pinecone, ServerlessSpec
    
    pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
    
    # List existing indexes first
    existing_indexes = pc.list_indexes()
    print("Existing indexes:", existing_indexes)
    
    index_name = "medical-bot"
    
    if index_name not in [idx.name for idx in existing_indexes]:
        print(f"Creating new index: {index_name}")
        pc.create_index(
            name=index_name,
            dimension=384,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        )
    else:
        print(f"Index {index_name} already exists")
    
    # Test connection to the index
    index = pc.Index(index_name)
    stats = index.describe_index_stats()
    print("Index stats:", stats)
    
except Exception as e:
    print("Pinecone error:", str(e))
    print("Error type:", type(e).__name__)

In [ ]:
!pip list | findstr pinecone

In [ ]:
import os
from dotenv import load_dotenv
  # must 
  # run first

load_dotenv()  # must run first

print(os.getenv("NVIDIA_API_KEY"))  # should show your key


In [ ]:
# Check what pinecone packages are installed
import subprocess
import sys

try:
    result = subprocess.run([sys.executable, "-m", "pip", "list"], 
                          capture_output=True, text=True, check=True)
    
    # Filter for pinecone packages
    pinecone_packages = [line for line in result.stdout.split('\n') 
                        if 'pinecone' in line.lower()]
    
    if pinecone_packages:
        print("Pinecone packages found:")
        for pkg in pinecone_packages:
            print(f"  {pkg}")
    else:
        print("No pinecone packages found")
        
except Exception as e:
    print(f"Error checking packages: {e}")

# Also try to import and check what we have
try:
    import pinecone
    print(f"\nPinecone module location: {pinecone.__file__}")
    
    # Check available attributes
    print("Available attributes:", [attr for attr in dir(pinecone) if not attr.startswith('_')])
    
except ImportError as e:
    print(f"Cannot import pinecone: {e}")
except Exception as e:
    print(f"Other error with pinecone: {e}")

In [ ]:
# Fix Pinecone version - uninstall old version and install new one
!pip uninstall pinecone pinecone-plugin-assistant pinecone-plugin-interface -y
!pip install "pinecone-client>=3.0.0"

In [ ]:
# Test the new Pinecone installation (run after kernel restart)
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables

try:
    from pinecone import Pinecone, ServerlessSpec
    print("✅ Pinecone imports successful!")
    
    # Test connection
    pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
    print("✅ Pinecone client initialized!")
    
    # List indexes
    indexes = pc.list_indexes()
    print("Available indexes:", indexes)
    
    # Check if medical-bot index exists, create if not
    index_name = "medical-bot"
    if not any(idx.name == index_name for idx in indexes):
        print(f"Creating index: {index_name}")
        pc.create_index(
            name=index_name,
            dimension=384,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        )
        print(f"✅ Index {index_name} created!")
    else:
        print(f"✅ Index {index_name} already exists!")
    
    # Test index connection
    index = pc.Index(index_name)
    stats = index.describe_index_stats()
    print("Index stats:", stats)
    
except ImportError as e:
    print("❌ Import error:", e)
    print("Make sure you've restarted the kernel after installing the new version")
except Exception as e:
    print("❌ Error:", e)

In [ ]:
# Fix Pinecone package name - uninstall pinecone-client and install pinecone
!pip uninstall pinecone-client -y
!pip install pinecone

# Clean NVIDIA-Only RAG Implementation
**Using NVIDIA's free `openai/gpt-oss-120b` model instead of OpenAI**

In [ ]:
# COMPLETE NVIDIA-ONLY RAG SETUP (No OpenAI needed)
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# ================================
# 1. Document Loading & Processing
# ================================
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

def load_and_process_documents():
    # Load PDFs
    loader = DirectoryLoader("Data/", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    
    # Split into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    
    print(f"Loaded {len(documents)} documents, split into {len(text_chunks)} chunks")
    return text_chunks

# Load documents
text_chunks = load_and_process_documents()

# Create embeddings (free local model)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("✅ Documents and embeddings ready!")

Loaded 637 documents, split into 5859 chunks


C:\Users\moina\AppData\Local\Temp\ipykernel_26984\2234481858.py:35: LangChainDeprecationWarning: The class `HuggingFaceHubEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpointEmbeddings``.
  embeddings = HuggingFaceHubEmbeddings(


ValidationError: 1 validation error for HuggingFaceHubEmbeddings
  Value error, Did not find huggingfacehub_api_token, please add an environment variable `HUGGINGFACEHUB_API_TOKEN` which contains it, or pass `huggingfacehub_api_token` as a named parameter. [type=value_error, input_value={'repo_id': 'sentence-tra...rgs': {'device': 'cpu'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
# ================================
# 2. Pinecone Vector Database Setup
# ================================
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "medical-bot"

# Create index if it doesn't exist
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,  # Matches HuggingFace embedding dimension
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"✅ Created new index: {index_name}")
else:
    print(f"✅ Using existing index: {index_name}")

# Connect to vector store
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name, 
    embedding=embeddings
)

# Create retriever
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print("✅ Pinecone vector store ready!")

c:\Users\moina\.conda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Using existing index: medical-bot
✅ Pinecone vector store ready!


In [5]:
# ================================
# 3. NVIDIA API Setup (FREE!)
# ================================
from openai import OpenAI

# NVIDIA API client using OpenAI-compatible interface
nvidia_client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.getenv("NVIDIA_API_KEY")
)

# Test NVIDIA API connection
def test_nvidia_api():
    try:
        response = nvidia_client.chat.completions.create(
            model="openai/gpt-oss-20b",  # FREE model
            messages=[{"role": "user", "content": "Say hello!"}],
            max_tokens=50,
            temperature=0.7
        )
        print("✅ NVIDIA API working!")
        print("Response:", response.choices[0].message.content)
        return True
    except Exception as e:
        print("❌ NVIDIA API Error:", str(e))
        return False

# Test the connection
test_nvidia_api()

✅ NVIDIA API working!
Response: 


True

In [6]:
# ================================
# 4. LangChain Integration with NVIDIA
# ================================
from typing import Any, List, Optional
from langchain_core.language_models.llms import LLM
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.outputs import LLMResult

class NvidiaLLM(LLM):
    """Custom LangChain LLM wrapper for NVIDIA API"""
    
    model: str = "openai/gpt-oss-20b"
    temperature: float = 0.4
    max_tokens: int = 500

    @property
    def _llm_type(self) -> str:
        return "nvidia-llm"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Make API call to NVIDIA"""
        try:
            completion = nvidia_client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=self.temperature,
                max_tokens=self.max_tokens,
            )
            return completion.choices[0].message.content
        except Exception as e:
            return f"Error: {str(e)}"

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> LLMResult:
        """Generate responses for multiple prompts"""
        responses = [self._call(p, stop=stop, run_manager=run_manager, **kwargs) for p in prompts]
        return LLMResult(generations=[[{"text": r}] for r in responses])

# Create the LLM instance
nvidia_llm = NvidiaLLM()

print("✅ NVIDIA LLM wrapper ready!")

✅ NVIDIA LLM wrapper ready!


In [7]:
# ================================
# 5. RAG Chain Setup
# ================================
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

# Create medical assistant prompt
system_prompt = (
    "You are a knowledgeable medical assistant. "
    "Use the following medical documentation to answer questions accurately. "
    "If you don't know the answer based on the provided context, say so clearly. "
    "Provide concise, helpful medical information. "
    "Always recommend consulting healthcare professionals for medical advice.\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Create the RAG chain
question_answer_chain = create_stuff_documents_chain(nvidia_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("✅ RAG chain ready with NVIDIA LLM!")

✅ RAG chain ready with NVIDIA LLM!


In [8]:
# ================================
# 6. Test the Complete System
# ================================

def ask_medical_question(question: str) -> str:
    """Ask a question to the medical chatbot"""
    try:
        response = rag_chain.invoke({"input": question})
        return response["answer"]
    except Exception as e:
        return f"Error: {str(e)}"

# Test questions
test_questions = [
    "how to cure Acne?"
]

print("🏥 Testing Medical Chatbot with NVIDIA's Free GPT-OSS-20B:")
print("=" * 60)

for i, question in enumerate(test_questions, 1):
    print(f"\n❓ Question {i}: {question}")
    print("-" * 40)
    answer = ask_medical_question(question)
    print(f"🤖 Answer: {answer}")
    print("-" * 40)

🏥 Testing Medical Chatbot with NVIDIA's Free GPT-OSS-20B:

❓ Question 1: how to cure Acne?
----------------------------------------
🤖 Answer: Error: Expecting value: line 1 column 1 (char 0)
----------------------------------------


In [18]:
# Testing

In [ ]:
# -------------------------------
# 1. Load & split documents
# -------------------------------
loader = DirectoryLoader("Data/", glob="*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents)

# -------------------------------
# 2. HF embeddings
# -------------------------------
embeddings = HuggingFaceEndpointEmbeddings(
    repo_id="sentence-transformers/all-MiniLM-L12-v2",
    huggingfacehub_api_token=HF_API_TOKEN
)

# -------------------------------
# 3. Pinecone
# -------------------------------
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medical-bot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})




C:\Users\moina\AppData\Local\Temp\ipykernel_26984\1336211573.py:43: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


Acidosis see Respiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when the
pores of the skin become clogged with oil, dead skin
cells, and bacteria.
Description
Acne vulgaris, t ...


In [29]:
# -------------------------------
# 4. Test
# -------------------------------
query = "how to cure Acne?"
docs = retriever.get_relevant_documents(query)
print(docs[0].page_content[:300], "...")

• shampoo often and wear hair off face
• eat a well-balanced diet, avoiding foods that trigger
flare-ups
• unless told otherwise, give dry pimples a limited
amount of sun exposur
• do not pick or squeeze blemishes
• reduce stress
Resources
BOOKS
Balch, James F., and Phyllis A. Balch. “The Disorders: ...
